<a href="https://colab.research.google.com/github/TanishqLambhate/Data-Science-Training/blob/pyspark_coding_challenge/Retail_Stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=8d88cca7b273d68eafa59bf5c0e872d774da328e58032a4f45849a92945b23c0
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
spark=SparkSession.builder.appName('Coding Challenge').getOrCreate()
csv_path="/content/Retail_Store.csv"
df=spark.read.csv(csv_path,header=True,inferSchema=True)
df.show()

+--------------+------------+-----------+-----+--------+----------+
|transaction_id|product_name|   category|price|quantity|sales_date|
+--------------+------------+-----------+-----+--------+----------+
|             1|       Apple|  Groceries|  0.5|      10|2023-09-01|
|             2|     T-shirt|   Clothing| 15.0|       2|2023-09-01|
|             3|    Notebook| Stationery|  2.0|       5|2023-09-02|
|             4|      Banana|  Groceries|  0.3|      12|2023-09-02|
|             5|      Laptop|Electronics|800.0|       1|2023-09-03|
|             6|       Pants|   Clothing| 25.0|       3|2023-09-03|
|             7|  Headphones|Electronics|100.0|       2|2023-09-04|
|             8|         Pen| Stationery|  1.0|      10|2023-09-04|
|             9|      Orange|  Groceries|  0.6|       8|2023-09-05|
|            10|    Sneakers|   Clothing| 50.0|       1|2023-09-05|
+--------------+------------+-----------+-----+--------+----------+



In [9]:
# 1. Calculate the Total Revenue per Category
# Group the data by category and calculate the total revenue generated by
# each category. (Hint: Multiply price by quantity for each
# transaction.)
df.groupBy('Category').agg(sum(col('Price')*col('Quantity')).alias('Total Revenue')).show()

# 2. Filter Transactions Where the Total Sales Amount is Greater Than $100
# Filter the dataset to show only transactions where the total sales
# amount (price * quantity) is greater than $100.
df.filter(col('Price')*col('Quantity')>100).show()

# 3. Find the Most Sold Product
# Identify the product with the highest total quantity sold across all
# transactions.
df.groupBy('product_name').agg(sum(col('quantity')).alias('Total Quantity')).orderBy(col('Total Quantity').desc()).show(1)

# 4. Calculate the Average Price per Product Category
# Group the data by category and calculate the average price of products
# in each category.
df.groupBy('Category').agg(avg(col('Price')).alias('Average Price')).show()

# 5. Find the Top 3 Highest Grossing Products
# Calculate the total revenue for each product and identify the top 3
# products that generated the most revenue.
df.groupBy('product_name').agg(sum(col('Price')*col('Quantity')).alias('Total Revenue')).orderBy(col('Total Revenue').desc()).show(3)

+-----------+------------------+
|   Category|     Total Revenue|
+-----------+------------------+
| Stationery|              20.0|
|  Groceries|13.399999999999999|
|Electronics|            1000.0|
|   Clothing|             155.0|
+-----------+------------------+

+--------------+------------+-----------+-----+--------+----------+
|transaction_id|product_name|   category|price|quantity|sales_date|
+--------------+------------+-----------+-----+--------+----------+
|             5|      Laptop|Electronics|800.0|       1|2023-09-03|
|             7|  Headphones|Electronics|100.0|       2|2023-09-04|
+--------------+------------+-----------+-----+--------+----------+

+------------+--------------+
|product_name|Total Quantity|
+------------+--------------+
|      Banana|            12|
+------------+--------------+
only showing top 1 row

+-----------+------------------+
|   Category|     Average Price|
+-----------+------------------+
| Stationery|               1.5|
|  Groceries|0.46666

In [15]:
# 6. Calculate the Total Number of Items Sold per Day
# Group the data by sales_date and calculate the total quantity of items
# sold for each day.
df.groupBy('sales_date').agg(sum(col('Quantity')).alias('Total Quantity')).show()

# 7. Identify the Product with the Lowest Price in Each Category
# For each category, identify the product with the lowest price.
window_spec=Window.partitionBy('Category').orderBy('Price')

df.withColumn('rank',row_number().over(window_spec)).filter(col('rank')==1).drop('rank').show()

# 8. Calculate the Total Revenue for Each Product
# Group the data by product_name and calculate the total revenue
# generated by each product.
df.groupBy('product_name').agg(sum(col('Price')*col('Quantity')).alias('Total Revenue')).show()

# 9. Find the Total Sales per Day for Each Category
# Group the data by sales_date and category to calculate the total sales
# for each category per day.
df.groupBy('sales_date','Category').agg(sum(col('Quantity')).alias('Total Quantity')).show()

# 10. Create a New Column for Discounted Price
# Add a new column called discounted_price that applies a 10% discount to
# the original price for each product ( price * 0.9 )
df.withColumn('discounted_price',col('Price')*0.9).show()


+----------+--------------+
|sales_date|Total Quantity|
+----------+--------------+
|2023-09-03|             4|
|2023-09-01|            12|
|2023-09-05|             9|
|2023-09-02|            17|
|2023-09-04|            12|
+----------+--------------+

+--------------+------------+-----------+-----+--------+----------+
|transaction_id|product_name|   category|price|quantity|sales_date|
+--------------+------------+-----------+-----+--------+----------+
|             2|     T-shirt|   Clothing| 15.0|       2|2023-09-01|
|             7|  Headphones|Electronics|100.0|       2|2023-09-04|
|             4|      Banana|  Groceries|  0.3|      12|2023-09-02|
|             8|         Pen| Stationery|  1.0|      10|2023-09-04|
+--------------+------------+-----------+-----+--------+----------+

+------------+------------------+
|product_name|     Total Revenue|
+------------+------------------+
|     T-shirt|              30.0|
|    Sneakers|              50.0|
|      Orange|               4.8